In [ ]:
import numpy as np
import pandas as pd
import itertools
import h5py
import matplotlib.pyplot as plt
from scipy import stats
from scipy import special
from scipy import integrate
from scipy import interpolate
from scipy import linalg
from scipy import signal
from scipy.optimize import curve_fit
import time
from pathlib import Path
import os
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['text.usetex'] = True

import sys
import mpld3

#mpld3.enable_notebook()

sys.path.insert(0, '..')
from modules import prh
from modules import regression as rg

In [ ]:
def get_season_masks(t):
    diffs = np.diff(t)
    
    season_ends = np.where(diffs >= diffs.mean() + 3*diffs.std())[0]
    season_begs = season_ends + 1
    
    season_begs = np.append(season_begs, 0)
    season_ends = np.append(season_ends, len(t)-1)
    
    season_ends.sort()
    season_begs.sort()
    
    masks = []
    
    for beg, end in zip(season_begs, season_ends):
        masks.append((t >= t[beg]) & (t <= t[end]))
    
    return masks
    

## Load data

In [ ]:
he_qso = pd.read_csv('../data/mindstep/HE_0047-1756.csv', delim_whitespace=True)

t = he_qso['MJD'].values
yA = he_qso['magAV'].values
yB = he_qso['magBV'].values
errA = he_qso['sigmaAV'].values
errB = he_qso['sigmaBV'].values

# drop nan

a_nan_mask = (1-np.isnan(yA)).astype(bool)
t = t[a_nan_mask]
yA = yA[a_nan_mask]
yB = yB[a_nan_mask]
errA = errA[a_nan_mask]
errB = errB[a_nan_mask]

season_masks = get_season_masks(t)

#plt.scatter(t, yA, label='all')
for i, mask in enumerate(season_masks):
    plt.scatter(t[mask], yA[mask], label='season %s' % (i+1))
plt.legend()

In [ ]:
kernel = ConstantKernel(2, (1e-3, 1e2)) * Matern(length_scale=200.0, length_scale_bounds=(1, 300), nu=1.5)

gpA = rg.fit_GP_to_lightcurve(t, yA, errA, kernel)
gpB = rg.fit_GP_to_lightcurve(t, yB, errB, kernel)

gp_step = 0.2
support = np.arange(t[0] - 5e0, t[-1] + 5e0, gp_step)

ypredA, sigmaA = gpA.predict(np.expand_dims(support, 1), return_std=True)
ypredB, sigmaB = gpB.predict(np.expand_dims(support, 1), return_std=True)

In [ ]:
CL = 0.95
fig = plt.figure()
plt.scatter(t, yA, label='magAV', color='b')
plt.scatter(t, yB, label='magBV', color='g')

plt.plot(support, ypredA, label='magAV GP prediction', color = 'b')
plt.plot(support, ypredB, label='magBV GP prediction', color = 'g')

plt.fill_between(support, ypredA - special.erfinv(CL)*sigmaA, ypredA + special.erfinv(CL)*sigmaA,
         alpha=.5, fc='b', ec='None', label=r'95\% confidence interval A')
plt.fill_between(support, ypredB - special.erfinv(CL)*sigmaB, ypredB + special.erfinv(CL)*sigmaB,
         alpha=.5, fc='g', ec='None', label=r'95\% confidence interval B')
plt.legend(fontsize=10)

plt.xlabel('t[MJD]')
plt.ylabel('mag')
plt.title('HE0047-1756 (MindStep data)')

fig.savefig('mindstep_gp_prediction.pdf')

In [ ]:
CL = 0.95
fig = plt.figure()
diff = ypredA - ypredB
sigma_diff = sigmaA + sigmaB

plt.scatter(t, yA-yB, color='blue', label='magAV - magBV (data)')
plt.plot(support, diff, label='magAV - magBV (predicted)', color='red')
plt.fill_between(support, 
                 diff - special.erfinv(CL)*sigma_diff, 
                 diff + special.erfinv(CL)*sigma_diff,
                 alpha=.5, fc='red', ec='None', label=r'95\% CL')
plt.legend()
plt.xlabel('t[MJD]')
fig.savefig('diff_pred.pdf')

In [ ]:
def numeric_derivative(f: np.ndarray, step: float) -> np.ndarray:
    return (f[1:] - f[:-1]) / step


def WAV(f, sigma, step):
    f_prime = numeric_derivative(f, step)
    weights = 2 / (sigma[:-1] + sigma[1:])
    WAV = np.dot(np.abs(f_prime), weights) / weights.sum()
    return WAV

In [ ]:
dt_min_days = 0
dt_max_days = 50

shift = np.arange(int(dt_min_days / gp_step),
                  int(dt_max_days / gp_step), 1)
win = len(shift)

In [ ]:
WAV_values = []
for i in shift:
    diff = ypredA[win:-win] - ypredB[win + i:-win + i]
    sigma_diff = sigmaA[win:-win] + sigmaB[win + i:-win + i]
    WAV_values.append(WAV(diff, sigma_diff, gp_step))
    
delay = np.argmin(WAV_values) * gp_step
print(delay)

In [ ]:
shift_dom = gp_step*shift
fig = plt.figure()
plt.plot(shift_dom, WAV_values, label='WAV loss function')
plt.xlabel('time lag[days]')
plt.axvline(x=7.6, color='red', label='Mindstep result (7.6 days)')
plt.legend()
fig.savefig('wav_loss.pdf')